In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer2 import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
locs += np.array([0,0,-0.70])
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Meshcat listening for connections at http://localhost:7001


In [5]:
visualizer.visualize_collision_constraint2d(factor = 1.2, num_points = 100)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
visualizer.meshcat2.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([0,0,1])))
visualizer.meshcat1.Set2dRenderMode(RigidTransform(RotationMatrix.MakeZRotation(0), np.array([1,0,0])))

## Set up the sliders so we can move the plant around manually

In [7]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

In [8]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


# Setup IRIS Options and Generate Regions

In [9]:
# Some examples of some seed points which give large regions using the
# non-linear programming approach to IRIS described in Algorithm 3
seed_points_q = np.array([   [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.4,-1.3]
                              ])
seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)


In [10]:
regions = []
ellipses = []



iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-4
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(2)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False

def promote_region_to_3d(region, width = 0.2):
    A = block_diag(region.A(), np.array([-1,1])[:, np.newaxis])
    b = np.append(region.b(), width*np.ones(2))

    return HPolyhedron(A,b)
    
regions_3d = []

for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    regions_3d.append(promote_region_to_3d(r))
    ellipses.append(r.MaximumVolumeInscribedEllipsoid())
    
# chosen for colorblind compatability
colors_dict = {
    0: (144,144,144), #grayish
    1:(30,136,229), # bluish
    2: (255, 193, 7), # gold
    3: (0, 140, 6), # green    
}
if do_viz:
    visualizer.plot_regions(regions,
                            ellipses=ellipses,
                            region_suffix='_original',
                            colors = list(colors_dict.values()),
                            wireframe = False,
                           opacity = 0.2)

In [11]:
r.IsBounded()

True

# Certify regions

In [12]:
cspace_free_region_certifier = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# make the solver verbose
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 1)

## first search for a minimal, uniform contraction of a proposed region which can be certified

In [13]:
i = 0
bisect_regions = []
bisect_solutions = []
directory = os.path.join(Path().absolute(), 
                                 f"pinball_iiwa_2d_bisection")
if not os.path.exists(directory):
        os.makedirs(directory)

do_cert = True
for i in range(len(regions)):
# for i in [1]:
    print(i)
    region_to_certify = regions[i]
    seed_point = seed_points[i,:]
    file_name = os.path.join(directory, f"seed_point{i}")
    if do_cert:

        binary_search_options = rational_forward_kinematics.BinarySearchOption()
        binary_search_options.epsilon_max = -1e-5 # it is very unlikely that we can find a uniform expansion of the current region
        binary_search_options.max_iters = 15
        binary_search_options.gap_tol = 1e-4
        # speed up the bisection search by taking non-uniform steps when possible
        binary_search_options.search_d = False
        # find the smallest e such that At <= b + e1 still contains our seed point.

        binary_search_options.epsilon_min = rational_forward_kinematics.FindEpsilonLower(region_to_certify.A(),
                                                                                         region_to_certify.b(),
                                                                                         limits_t[0], limits_t[1],
                                                                                         seed_point)
        #use as many threads as possible to speed up computation
    #     binary_search_options.num_threads = 1

        certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(
                                                                         q_star[:2],
                                                                         filtered_collision_pairs,
                                                                         region_to_certify.A(),
                                                                         region_to_certify.b(),
                                                                         binary_search_options, 
                                                                         solver_options,
                                                                         seed_point)

        if not os.path.exists(directory):
            os.makedirs(directory)

        rational_forward_kinematics.WriteCspacePolytopeToFile(certified_region_contraction_solution, plant,
                                                                      inspector,                            
                                                                      file_name,
                                                              9)
    else:
        certified_region_contraction_solution = rational_forward_kinematics.ReadCspacePolytopeFromFile(file_name, cspace_free_region_certifier)


    bisect_solutions.append(certified_region_contraction_solution)
    certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
                                               certified_region_contraction_solution.d)

    bisect_regions.append(certified_region_contraction)
    time.sleep(1)

    
if do_viz:
    visualizer.plot_regions(bisect_regions, ellipses=None,
                            region_suffix='_certified_region_contraction',
                            wireframe = False,
                           colors = list(colors_dict.values()),
                           opacity = 0.2)
visualizer.certified_region_solution_list = bisect_solutions
visualizer.color_dict = {i: tuple(val/255 for val in c) for i, c in colors_dict.items()}

0


INFO:drake:Lagrangian SOS takes 0.7003300189971924 seconds
INFO:drake:max(power(det(P), 1/2))=0.2385154924627056, solver_time 0.029612064361572266


1


INFO:drake:Lagrangian SOS takes 0.3206288814544678 seconds
INFO:drake:max(power(det(P), 1/2))=0.12499845351885229, solver_time 0.006716012954711914
INFO:drake:Lagrangian SOS takes 0.31062889099121094 seconds
INFO:drake:max(power(det(P), 1/2))=0.14258707297241818, solver_time 0.00609898567199707
INFO:drake:epsilon=-0.006375108966417737 is feasible
INFO:drake:reset eps_min=0.0, eps_max=0.006365108966417732
INFO:drake:Lagrangian SOS takes 0.31740617752075195 seconds
INFO:drake:max(power(det(P), 1/2))=0.1506978962566649, solver_time 0.014928817749023438
INFO:drake:epsilon=0.003182554483208866 is feasible
INFO:drake:reset eps_min=0.0, eps_max=0.0031825544832089214
INFO:drake:Lagrangian SOS takes 0.35380101203918457 seconds
INFO:drake:max(power(det(P), 1/2))=0.1546157881837404, solver_time 0.00815582275390625
INFO:drake:epsilon=0.0015912772416044607 is feasible
INFO:drake:reset eps_min=0.0, eps_max=0.0015912772416044607
INFO:drake:Lagrangian SOS takes 0.38993287086486816 seconds
INFO:drake:m

2


INFO:drake:Lagrangian SOS takes 0.6591789722442627 seconds
INFO:drake:max(power(det(P), 1/2))=0.409674763277814, solver_time 0.006042003631591797
INFO:drake:Lagrangian SOS takes 0.39054298400878906 seconds


3


INFO:drake:max(power(det(P), 1/2))=0.403230266799232, solver_time 0.004277944564819336


In [14]:
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra
from pydrake.all import set_log_level
from pydrake.all import logging
log = logging.getLogger("drake")
log.setLevel("DEBUG")

start_point = Ratfk.ComputeTValue(np.array([0,0.1]), q_star)
end_point = Ratfk.ComputeTValue(np.array([-1.0,0]), q_star)
# end_point = Ratfk.ComputeTValue(np.array([0,0.5]), q_star)

spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start_point, end_point, regions)
spp.set_max_velocity([.8, .8])
spp.set_extra_control_points_per_region(10)

In [15]:

traj = spp.Solve()
visualizer.draw_traj_s_space(traj, 100)

In [23]:
self = visualizer
visualizer.certified_region_solution_list = []# bisect_solutions # []# bisect_solutions

# for i in range(len(seed_points)):
#     self.meshcat2.SetProperty(f"/iris/regions_certified_region_contraction/{i}", "visible", True)
#     time.sleep(1)

substeps = 100
runtime = 2*substeps

tmp = visualizer.vis.StartRecording()
# visualizer.vis2.StartRecording()
visualizer.animate_s(traj, substeps, runtime)

visualizer.vis.PublishRecording()
# visualizer.vis2.PublishRecording()
# visualizer.vis.StopRecording()
# visualizer.vis2.StopRecording()


In [17]:
sliders = []
sliders.append(widgets.FloatSlider(min=traj.start_time(), max=traj.end_time(), value=0, description='t'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change_traj(change):
    q = self.forward_kin.ComputeQValue(traj.value(change['new']), self.q_star)
    visualizer.showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(handle_slider_change_traj, names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='t', max=3.8432898411962904)

In [22]:
for i in range(len(seed_points)):
    visualizer.meshcat1.Delete(f"/planes/region{i}")

In [27]:
from pydrake.all import BsplineBasis, BsplineTrajectory, VPolytope, HPolyhedron, Variable, PiecewisePolynomial

visualizer.certified_region_solution_list = bisect_solutions


verts_q = np.array([
    [-1.1, 0],
    [0.6, -1.7],
    [-0.4,-0.4],
    [-1.0,-1.6]
]).T
verts_q = np.concatenate([verts_q, verts_q[:,0][:,np.newaxis]], axis = 1)

verts = np.array([Ratfk.ComputeTValue(v.copy(), q_star) for v in verts_q.T]).T


times = np.linspace(0, 5, verts.shape[1])

traj2 = PiecewisePolynomial.FirstOrderHold(times, verts)

substeps = 300
runtime = 2*substeps

visualizer.animate_s(traj2, substeps, runtime, idx_list = [2])

    


In [100]:
# loop
steps = substeps

idx = 0
going_fwd = True
time_points = np.linspace(0, traj2.end_time(), steps)

for _ in range(runtime):
    # print(idx)
    s = traj2.value(time_points[idx])
    print(s)
    print()
    q = self.forward_kin.ComputeQValue(s, self.q_star)
    self.showres(q)
    if going_fwd:
        if idx + 1 < steps:
            idx += 1
        else:
            going_fwd = False
            idx -= 1
    else:
        if idx - 1 >= 0:
            idx -= 1
        else:
            going_fwd = True
            idx += 1

[[-0.31955625]
 [-0.63978599]]

[[-0.3183805 ]
 [-0.63978599]]

[[-0.31720475]
 [-0.63978599]]

[[-0.316029  ]
 [-0.63978599]]

[[-0.31485325]
 [-0.63978599]]

[[-0.3136775 ]
 [-0.63978599]]

[[-0.31250175]
 [-0.63978599]]

[[-0.311326  ]
 [-0.63978599]]

[[-0.31015024]
 [-0.63978599]]

[[-0.30897449]
 [-0.63978599]]

[[-0.30779874]
 [-0.63978599]]

[[-0.30662299]
 [-0.63978599]]

[[-0.30544724]
 [-0.63978599]]

[[-0.30427149]
 [-0.63978599]]

[[-0.30309574]
 [-0.63978599]]

[[-0.30191999]
 [-0.63978599]]

[[-0.30074424]
 [-0.63978599]]

[[-0.29956848]
 [-0.63978599]]

[[-0.29839273]
 [-0.63978599]]

[[-0.29721698]
 [-0.63978599]]

[[-0.29604123]
 [-0.63978599]]

[[-0.29486548]
 [-0.63978599]]

[[-0.29368973]
 [-0.63978599]]

[[-0.29251398]
 [-0.63978599]]

[[-0.29133823]
 [-0.63978599]]

[[-0.29016247]
 [-0.63978599]]

[[-0.28898672]
 [-0.63978599]]

[[-0.28781097]
 [-0.63978599]]

[[-0.28663522]
 [-0.63978599]]

[[-0.28545947]
 [-0.63978599]]

[[-0.28428372]
 [-0.63978599]]

[[-0.283

[[-0.0079822 ]
 [-0.63978599]]

[[-0.00680645]
 [-0.63978599]]

[[-0.0056307 ]
 [-0.63978599]]

[[-0.00445494]
 [-0.63978599]]

[[-0.00327919]
 [-0.63978599]]

[[-0.00210344]
 [-0.63978599]]

[[-0.00092769]
 [-0.63978599]]

[[ 2.48059867e-04]
 [-6.39785995e-01]]

[[ 0.00142381]
 [-0.63978599]]

[[ 0.00259956]
 [-0.63978599]]

[[ 0.00377531]
 [-0.63978599]]

[[ 0.00495106]
 [-0.63978599]]

[[ 0.00612682]
 [-0.63978599]]

[[ 0.00730257]
 [-0.63978599]]

[[ 0.00847832]
 [-0.63978599]]

[[ 0.00965407]
 [-0.63978599]]

[[ 0.01082982]
 [-0.63978599]]

[[ 0.01200557]
 [-0.63978599]]

[[ 0.01318132]
 [-0.63978599]]

[[ 0.01435707]
 [-0.63978599]]

[[ 0.01553282]
 [-0.63978599]]

[[ 0.01670858]
 [-0.63978599]]

[[ 0.01788433]
 [-0.63978599]]

[[ 0.01906008]
 [-0.63978599]]

[[ 0.02023583]
 [-0.63978599]]

[[ 0.02141158]
 [-0.63978599]]

[[ 0.02258733]
 [-0.63978599]]

[[ 0.02376308]
 [-0.63978599]]

[[ 0.02493883]
 [-0.63978599]]

[[ 0.02611459]
 [-0.63978599]]

[[ 0.02729034]
 [-0.63978599]]



[[-0.26916687]
 [-0.03372092]]

[[-0.27034262]
 [-0.03214906]]

[[-0.27151837]
 [-0.0305772 ]]

[[-0.27269412]
 [-0.02900534]]

[[-0.27386987]
 [-0.02743348]]

[[-0.27504562]
 [-0.02586161]]

[[-0.27622137]
 [-0.02428975]]

[[-0.27739712]
 [-0.02271789]]

[[-0.27857287]
 [-0.02114603]]

[[-0.27974863]
 [-0.01957417]]

[[-0.28092438]
 [-0.0180023 ]]

[[-0.28210013]
 [-0.01643044]]

[[-0.28327588]
 [-0.01485858]]

[[-0.28445163]
 [-0.01328672]]

[[-0.28562738]
 [-0.01171486]]

[[-0.28680313]
 [-0.01014299]]

[[-0.28797888]
 [-0.00857113]]

[[-0.28915463]
 [-0.00699927]]

[[-0.29033038]
 [-0.00542741]]

[[-0.29150614]
 [-0.00385555]]

[[-0.29268189]
 [-0.00228368]]

[[-0.29385764]
 [-0.00071182]]

[[-0.29503339]
 [ 0.00086004]]

[[-0.29620914]
 [ 0.0024319 ]]

[[-0.29738489]
 [ 0.00400376]]

[[-0.29856064]
 [ 0.00557563]]

[[-0.29973639]
 [ 0.00714749]]

[[-0.30091214]
 [ 0.00871935]]

[[-0.30208789]
 [ 0.01029121]]

[[-0.30326365]
 [ 0.01186307]]

[[-0.3044394 ]
 [ 0.01343494]]

[[-0.305

 [-0.05970625]]

[[-0.16607115]
 [-0.06007357]]

[[-0.16546959]
 [-0.06044088]]

[[-0.16486802]
 [-0.0608082 ]]

[[-0.16426646]
 [-0.06117552]]

[[-0.16366489]
 [-0.06154283]]

[[-0.16306333]
 [-0.06191015]]

[[-0.16246176]
 [-0.06227746]]

[[-0.1618602 ]
 [-0.06264478]]

[[-0.16125863]
 [-0.0630121 ]]

[[-0.16065707]
 [-0.06337941]]

[[-0.1600555 ]
 [-0.06374673]]

[[-0.15945394]
 [-0.06411405]]

[[-0.15885237]
 [-0.06448136]]

[[-0.15825081]
 [-0.06484868]]

[[-0.15764924]
 [-0.065216  ]]

[[-0.15704768]
 [-0.06558331]]

[[-0.15644611]
 [-0.06595063]]

[[-0.15584454]
 [-0.06631795]]

[[-0.15524298]
 [-0.06668526]]

[[-0.15464141]
 [-0.06705258]]

[[-0.15403985]
 [-0.06741989]]

[[-0.15343828]
 [-0.06778721]]

[[-0.15283672]
 [-0.06815453]]

[[-0.15223515]
 [-0.06852184]]

[[-0.15163359]
 [-0.06888916]]

[[-0.15103202]
 [-0.06925648]]

[[-0.15043046]
 [-0.06962379]]

[[-0.14982889]
 [-0.06999111]]

[[-0.14922733]
 [-0.07035843]]

[[-0.14862576]
 [-0.07072574]]

[[-0.1480242 ]
 [-0.071

 [-0.11608349]]

[[-0.06635672]
 [-0.11599435]]

[[-0.06640894]
 [-0.11590521]]

[[-0.06646116]
 [-0.11581606]]

[[-0.06651338]
 [-0.11572692]]

[[-0.0665656 ]
 [-0.11563778]]

[[-0.06661782]
 [-0.11554864]]

[[-0.06667004]
 [-0.11545949]]

[[-0.06672226]
 [-0.11537035]]

[[-0.06677448]
 [-0.11528121]]

[[-0.0668267 ]
 [-0.11519207]]

[[-0.06687892]
 [-0.11510293]]

[[-0.06693114]
 [-0.11501378]]

[[-0.06698336]
 [-0.11492464]]

[[-0.06703558]
 [-0.1148355 ]]

[[-0.0670878 ]
 [-0.11474636]]

[[-0.06714002]
 [-0.11465721]]

[[-0.06719224]
 [-0.11456807]]

[[-0.06724446]
 [-0.11447893]]

[[-0.06729668]
 [-0.11438979]]

[[-0.0673489 ]
 [-0.11430065]]

[[-0.06740112]
 [-0.1142115 ]]

[[-0.06745334]
 [-0.11412236]]

[[-0.06750556]
 [-0.11403322]]

[[-0.06755778]
 [-0.11394408]]

[[-0.06761   ]
 [-0.11385493]]

[[-0.06766222]
 [-0.11376579]]

[[-0.06771444]
 [-0.11367665]]

[[-0.06776666]
 [-0.11358751]]

[[-0.06781888]
 [-0.11349837]]

[[-0.0678711 ]
 [-0.11340922]]

[[-0.06792332]
 [-0.113

...

 [-0.10788241]]

[[-0.07105651]
 [-0.10797155]]

[[-0.07100429]
 [-0.10806069]]

[[-0.07095207]
 [-0.10814983]]

[[-0.07089985]
 [-0.10823898]]

[[-0.07084763]
 [-0.10832812]]

[[-0.07079541]
 [-0.10841726]]

[[-0.07074319]
 [-0.1085064 ]]

[[-0.07069097]
 [-0.10859554]]

[[-0.07063875]
 [-0.10868469]]

[[-0.07058653]
 [-0.10877383]]

[[-0.07053431]
 [-0.10886297]]

[[-0.07048209]
 [-0.10895211]]

[[-0.07042987]
 [-0.10904125]]

[[-0.07037765]
 [-0.1091304 ]]

[[-0.07032543]
 [-0.10921954]]

[[-0.07027321]
 [-0.10930868]]

[[-0.07022099]
 [-0.10939782]]

[[-0.07016877]
 [-0.10948697]]

[[-0.07011655]
 [-0.10957611]]

[[-0.07006433]
 [-0.10966525]]

[[-0.07001211]
 [-0.10975439]]

[[-0.06995989]
 [-0.10984353]]

[[-0.06990767]
 [-0.10993268]]

[[-0.06985545]
 [-0.11002182]]

[[-0.06980323]
 [-0.11011096]]

[[-0.06975101]
 [-0.1102001 ]]

[[-0.06969879]
 [-0.11028925]]

[[-0.06964657]
 [-0.11037839]]

[[-0.06959435]
 [-0.11046753]]

[[-0.06954213]
 [-0.11055667]]

[[-0.06948991]
 [-0.110

 [-0.09423399]]

[[-0.11072716]
 [-0.09386668]]

[[-0.11132873]
 [-0.09349936]]

[[-0.11193029]
 [-0.09313204]]

[[-0.11253186]
 [-0.09276473]]

[[-0.11313342]
 [-0.09239741]]

[[-0.11373499]
 [-0.09203009]]

[[-0.11433655]
 [-0.09166278]]

[[-0.11493812]
 [-0.09129546]]

[[-0.11553968]
 [-0.09092814]]

[[-0.11614125]
 [-0.09056083]]

[[-0.11674281]
 [-0.09019351]]

[[-0.11734438]
 [-0.0898262 ]]

[[-0.11794594]
 [-0.08945888]]

[[-0.11854751]
 [-0.08909156]]

[[-0.11914907]
 [-0.08872425]]

[[-0.11975064]
 [-0.08835693]]

[[-0.1203522 ]
 [-0.08798961]]

[[-0.12095377]
 [-0.0876223 ]]

[[-0.12155533]
 [-0.08725498]]

[[-0.1221569 ]
 [-0.08688766]]

[[-0.12275846]
 [-0.08652035]]

[[-0.12336003]
 [-0.08615303]]

[[-0.12396159]
 [-0.08578571]]

[[-0.12456316]
 [-0.0854184 ]]

[[-0.12516472]
 [-0.08505108]]

[[-0.12576629]
 [-0.08468377]]

[[-0.12636785]
 [-0.08431645]]

[[-0.12696942]
 [-0.08394913]]

[[-0.12757098]
 [-0.08358182]]

[[-0.12817255]
 [-0.0832145 ]]

[[-0.12877411]
 [-0.082


[[-0.31737266]
 [ 0.03072542]]

[[-0.31619691]
 [ 0.02915356]]

[[-0.31502116]
 [ 0.02758169]]

[[-0.31384541]
 [ 0.02600983]]

[[-0.31266965]
 [ 0.02443797]]

[[-0.3114939 ]
 [ 0.02286611]]

[[-0.31031815]
 [ 0.02129425]]

[[-0.3091424 ]
 [ 0.01972238]]

[[-0.30796665]
 [ 0.01815052]]

[[-0.3067909 ]
 [ 0.01657866]]

[[-0.30561515]
 [ 0.0150068 ]]

[[-0.3044394 ]
 [ 0.01343494]]

[[-0.30326365]
 [ 0.01186307]]

[[-0.30208789]
 [ 0.01029121]]

[[-0.30091214]
 [ 0.00871935]]

[[-0.29973639]
 [ 0.00714749]]

[[-0.29856064]
 [ 0.00557563]]

[[-0.29738489]
 [ 0.00400376]]

[[-0.29620914]
 [ 0.0024319 ]]

[[-0.29503339]
 [ 0.00086004]]

[[-0.29385764]
 [-0.00071182]]

[[-0.29268189]
 [-0.00228368]]

[[-0.29150614]
 [-0.00385555]]

[[-0.29033038]
 [-0.00542741]]

[[-0.28915463]
 [-0.00699927]]

[[-0.28797888]
 [-0.00857113]]

[[-0.28680313]
 [-0.01014299]]

[[-0.28562738]
 [-0.01171486]]

[[-0.28445163]
 [-0.01328672]]

[[-0.28327588]
 [-0.01485858]]

[[-0.28210013]
 [-0.01643044]]

[[-0.28

 [-0.40468035]]

[[ 0.00948612]
 [-0.40625221]]

[[ 0.01066188]
 [-0.40782408]]

[[ 0.01183763]
 [-0.40939594]]

[[ 0.01301338]
 [-0.4109678 ]]

[[ 0.01418913]
 [-0.41253966]]

[[ 0.01536488]
 [-0.41411152]]

[[ 0.01654063]
 [-0.41568339]]

[[ 0.01771638]
 [-0.41725525]]

[[ 0.01889213]
 [-0.41882711]]

[[ 0.02006788]
 [-0.42039897]]

[[ 0.02124363]
 [-0.42197083]]

[[ 0.02241939]
 [-0.4235427 ]]

[[ 0.02359514]
 [-0.42511456]]

[[ 0.02477089]
 [-0.42668642]]

[[ 0.02594664]
 [-0.42825828]]

[[ 0.02712239]
 [-0.42983014]]

[[ 0.02829814]
 [-0.43140201]]

[[ 0.02947389]
 [-0.43297387]]

[[ 0.03064964]
 [-0.43454573]]

[[ 0.03182539]
 [-0.43611759]]

[[ 0.03300115]
 [-0.43768945]]

[[ 0.0341769 ]
 [-0.43926132]]

[[ 0.03535265]
 [-0.44083318]]

[[ 0.0365284 ]
 [-0.44240504]]

[[ 0.03770415]
 [-0.4439769 ]]

[[ 0.0388799 ]
 [-0.44554876]]

[[ 0.04005565]
 [-0.44712063]]

[[ 0.0412314 ]
 [-0.44869249]]

[[ 0.04240715]
 [-0.45026435]]

[[ 0.0435829 ]
 [-0.45183621]]

[[ 0.04475866]
 [-0.453

[[ 0.02141158]
 [-0.63978599]]

[[ 0.02023583]
 [-0.63978599]]

[[ 0.01906008]
 [-0.63978599]]

[[ 0.01788433]
 [-0.63978599]]

[[ 0.01670858]
 [-0.63978599]]

[[ 0.01553282]
 [-0.63978599]]

[[ 0.01435707]
 [-0.63978599]]

[[ 0.01318132]
 [-0.63978599]]

[[ 0.01200557]
 [-0.63978599]]

[[ 0.01082982]
 [-0.63978599]]

[[ 0.00965407]
 [-0.63978599]]

[[ 0.00847832]
 [-0.63978599]]

[[ 0.00730257]
 [-0.63978599]]

[[ 0.00612682]
 [-0.63978599]]

[[ 0.00495106]
 [-0.63978599]]

[[ 0.00377531]
 [-0.63978599]]

[[ 0.00259956]
 [-0.63978599]]

[[ 0.00142381]
 [-0.63978599]]

[[ 2.48059867e-04]
 [-6.39785995e-01]]

[[-0.00092769]
 [-0.63978599]]

[[-0.00210344]
 [-0.63978599]]

[[-0.00327919]
 [-0.63978599]]

[[-0.00445494]
 [-0.63978599]]

[[-0.0056307 ]
 [-0.63978599]]

[[-0.00680645]
 [-0.63978599]]

[[-0.0079822 ]
 [-0.63978599]]

[[-0.00915795]
 [-0.63978599]]

[[-0.0103337 ]
 [-0.63978599]]

[[-0.01150945]
 [-0.63978599]]

[[-0.0126852 ]
 [-0.63978599]]

[[-0.01386095]
 [-0.63978599]]



In [124]:
visualizer.showres(verts_q[:,0])

In [119]:
visualizer.showres_s(verts[:,0])

In [125]:
visualizer.showres(np.array([-1.1,0]))

In [121]:
verts_q[:,0]

array([-0.61860073, -1.13832271])